In [1]:
from datetime import datetime
import pandas as pd

import duckdb

In [2]:
duckdb.query(
    """

    """
)      #.fetchdf()는 이 sql문을 df로 저장하는 코드

# 데이터 수집

### 종목 기본정보

In [3]:
import requests
import pandas as pd

url = 'https://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

headers = {
    'User-Agent': 'Mozilla/5.0',
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201'
}

payload = {
    'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',  #핵심 API!
    'mktId': 'ALL',   # STK: 코스피, KSQ: 코스닥, ALL: 전체
    'share': '1',
    'csvxls_isNo': 'false'
}

res = requests.post(url, headers=headers, data=payload)
json_data = res.json()
kor_info_raw = pd.DataFrame(json_data['OutBlock_1'])

# 숫자로 변환
cols_to_convert = ['PARVAL', 'LIST_SHRS']  # 변환할 컬럼 목록
for col in cols_to_convert:
    kor_info_raw[col] = kor_info_raw[col].astype(str).str.replace(',', '', regex=False)
    kor_info_raw[col] = pd.to_numeric(kor_info_raw[col], errors='coerce')


kor_info = duckdb.query(
    """
    select
        MKT_TP_NM as 시장구분,
        ISU_SRT_CD as 티커,
        ISU_ABBRV as 종목명,
        STRPTIME(LIST_DD, '%Y/%m/%d') AS 상장일,
        KIND_STKCERT_TP_NM as 증권구분,
        PARVAL as 액면가,
        LIST_SHRS as 상장주식수
    from kor_info_raw
    """
).fetchdf()

kor_info.head()

,시장구분,티커,종목명,상장일,증권구분,액면가,상장주식수
0,KOSDAQ,098120,마이크로컨텍솔,2008-09-23,보통주,500.0,8312766
1,KOSDAQ GLOBAL,009520,포스코엠텍,1997-11-10,보통주,500.0,41642703
2,KOSPI,095570,AJ네트웍스,2015-08-21,보통주,1000.0,45252759
3,KOSPI,006840,AK홀딩스,1999-08-11,보통주,5000.0,13247561
4,KOSPI,282330,BGF리테일,2017-12-08,보통주,1000.0,17283906


### 종목 시세 - 실시간(20min) or 특정날짜

In [70]:
# 실시간
# today_str = datetime.today().strftime('%Y%m%d')

# 특정 조회날짜
today_str = 20250609

In [57]:
import requests
import pandas as pd

# 요청 URL
url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

# 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201'
}

# POST 요청의 form data (payload)
payload = {
    'bld': 'dbms/MDC/STAT/standard/MDCSTAT01501',  # 전종목 시세
    'locale': 'ko_KR',
    'mktId': 'ALL',             # 코스피: STK, 코스닥: KSQ, ALL로 하면 전체
    'trdDd': today_str,        # 특정 날짜
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false'
}

res = requests.post(url, headers=headers, data=payload)
data_json = res.json()
kor_price_raw = pd.DataFrame(data_json['OutBlock_1'])

# 숫자로 변환
exclude_cols = ['ISU_SRT_CD', 'ISU_CD', 'ISU_ABBRV', 'MKT_NM', 'SECT_TP_NM', 'MKT_ID'] # 제외할 칼럼 목록
for col in kor_price_raw.select_dtypes(include='object').columns:
    if col not in exclude_cols:
        kor_price_raw[col] = kor_price_raw[col].str.replace(',', '', regex=False)
        kor_price_raw[col] = pd.to_numeric(kor_price_raw[col], errors='coerce')


kor_price = duckdb.query(
    """
    select
        MKT_NM as 시장구분,
        ISU_SRT_CD as 티커,
        ISU_ABBRV as 종목명,
        TDD_CLSPRC as 종가,
        CMPPREVDD_PRC as 등락,
        FLUC_RT as 등락률,
        TDD_OPNPRC as 시가,
        TDD_HGPRC as 고가,
        TDD_LWPRC as 저가,
        ACC_TRDVOL as 거래량,
        ACC_TRDVAL as 거래대금,
        MKTCAP as 시가총액,
        LIST_SHRS as 상장주식수
    from kor_price_raw
    """
).fetchdf()

kor_price.head()

,시장구분,티커,종목명,종가,등락,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,KOSDAQ,060310,3S,2225,30,1.37,2200,2235,2180,301535,669610813,118056364000,53059040
1,KOSPI,095570,AJ네트웍스,4000,35,0.88,3970,4060,3970,408204,1639838300,181011036000,45252759
2,KOSPI,006840,AK홀딩스,12160,110,0.91,12240,12500,12110,19153,235802635,161090341760,13247561
3,KOSDAQ,054620,APS,5660,-40,-0.70,5700,5820,5500,36410,206748580,112601290860,19894221
4,KOSDAQ,265520,AP시스템,17290,260,1.53,17270,17410,17100,73220,1263152850,264215769090,15281421


---

In [58]:
duckdb.query(
    """
    select i.시장구분, i.티커, i.종목명, 종가, 등락률, 시가총액
    from kor_price p join kor_info i using(티커)
    where i.종목명 = '카카오페이'
    """
).fetchdf()

,시장구분,티커,종목명,종가,등락률,시가총액
0,KOSPI,377300,카카오페이,49500,29.92,6667328398500


In [60]:
duckdb.query(
    """
    select i.시장구분, i.티커, i.종목명, 종가, 등락률, 시가총액
    from kor_price p join kor_info i using(티커)
    order by 시가총액 desc
    """
).fetchdf()

,시장구분,티커,종목명,종가,등락률,시가총액
0,KOSPI,005930,삼성전자,59800,1.18,353994347735600
1,KOSPI,000660,SK하이닉스,229000,2.00,166712541585000
2,KOSPI,207940,삼성바이오로직스,1033000,1.37,73522742000000
3,KOSPI,373220,LG에너지솔루션,285000,-2.06,66690000000000
4,KOSPI,105560,KB금융,110700,4.14,42227854802100
...,...,...,...,...,...,...
2861,KONEX,288490,나라소프트,105,0.00,1834515585
2862,KONEX,236030,씨알푸드,897,-0.22,1823898804
2863,KONEX,245450,씨앤에스링크,1000,0.00,1579960000
2864,KONEX,308700,테크엔,199,0.00,1472600000


In [69]:
duckdb.query(
    """
    SELECT *
    FROM kor_price
    where 시장구분 = 'KOSPI'
    """
).fetchdf()

,시장구분,티커,종목명,종가,등락,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,KOSPI,095570,AJ네트웍스,4000,35,0.88,3970,4060,3970,408204,1639838300,181011036000,45252759
1,KOSPI,006840,AK홀딩스,12160,110,0.91,12240,12500,12110,19153,235802635,161090341760,13247561
2,KOSPI,027410,BGF,4470,170,3.95,4355,4525,4355,689130,3073814179,427854055770,95716791
3,KOSPI,282330,BGF리테일,123900,12400,11.12,112300,125500,112100,291821,35482256350,2141475953400,17283906
4,KOSPI,138930,BNK금융지주,11250,80,0.72,11200,11480,11200,1242646,14116730540,3581814588750,318383519
...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,KOSPI,079980,휴비스,3015,-25,-0.82,3040,3115,2990,99251,300359148,104017500000,34500000
958,KOSPI,005010,휴스틸,4980,115,2.36,4960,5100,4930,689905,3451215709,279816613500,56188075
959,KOSPI,000540,흥국화재,4160,-15,-0.36,4200,4350,4120,326666,1381249178,267249403200,64242645
960,KOSPI,000545,흥국화재우,7620,10,0.13,7610,7620,7510,26491,200514400,5852160000,768000


In [30]:
import requests
import zipfile
import io
import pandas as pd

api_key = '3adc2cc74bd0f5806d6364d34476a332aa420bd5'

# DART에서 전체 기업 코드 ZIP 다운로드
url = f'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}'
res = requests.get(url)
with zipfile.ZipFile(io.BytesIO(res.content)) as z:
    xml_content = z.read(z.namelist()[0])

# XML → DataFrame 변환
import xml.etree.ElementTree as ET
tree = ET.fromstring(xml_content)
corp_list = []
for child in tree:
    corp = {item.tag: item.text for item in child}
    corp_list.append(corp)
corp_df = pd.DataFrame(corp_list)

# 예: 삼성전자 찾기
corp_code = corp_df[corp_df['corp_name'] == 'SAMG엔터']['corp_code'].values[0]

In [31]:
# 사업보고서 (2023년 기준)
url = f'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json'
params = {
    'crtfc_key': api_key,
    'corp_code': corp_code,
    'bsns_year': '2023',
    'reprt_code': '11011',  # 11011: 사업보고서, 11012: 1Q, 11013: 반기, 11014: 3Q
    'fs_div': 'CFS'         # 연결: CFS / 개별: OFS
}

res = requests.get(url, params=params)
data = res.json()

df_fnl = pd.DataFrame(data['list'])
df_fnl

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,...,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord,currency
0,20240318000675,11011,2023,01082418,419530,CFS,연결재무제표,BS,재무상태표,유동자산,...,2023.12.31 현재,"60,626,359,363",제 23 기,2022.12.31 현재,"54,719,246,284",제 22 기,2021.12.31 현재,"23,667,936,467",1,KRW
1,20240318000675,11011,2023,01082418,419530,CFS,연결재무제표,BS,재무상태표,비유동자산,...,2023.12.31 현재,"39,508,123,335",제 23 기,2022.12.31 현재,"21,961,782,500",제 22 기,2021.12.31 현재,"14,815,979,715",3,KRW
2,20240318000675,11011,2023,01082418,419530,CFS,연결재무제표,BS,재무상태표,자산총계,...,2023.12.31 현재,"100,134,482,698",제 23 기,2022.12.31 현재,"76,681,028,784",제 22 기,2021.12.31 현재,"38,483,916,182",5,KRW
3,20240318000675,11011,2023,01082418,419530,CFS,연결재무제표,BS,재무상태표,유동부채,...,2023.12.31 현재,"21,218,756,805",제 23 기,2022.12.31 현재,"26,191,891,492",제 22 기,2021.12.31 현재,"18,416,975,776",7,KRW
4,20240318000675,11011,2023,01082418,419530,CFS,연결재무제표,BS,재무상태표,비유동부채,...,2023.12.31 현재,"31,824,206,962",제 23 기,2022.12.31 현재,"3,400,441,854",제 22 기,2021.12.31 현재,"42,742,812,301",9,KRW
5,20240318000675,11011,2023,01082418,419530,CFS,연결재무제표,BS,재무상태표,부채총계,...,2023.12.31 현재,"53,042,963,767",제 23 기,2022.12.31 현재,"29,592,333,346",제 22 기,2021.12.31 현재,"61,159,788,077",11,KRW
6,20240318000675,11011,2023,01082418,419530,CFS,연결재무제표,BS,재무상태표,자본금,...,2023.12.31 현재,"4,295,465,000",제 23 기,2022.12.31 현재,"4,295,465,000",제 22 기,2021.12.31 현재,"1,335,200,000",13,KRW
7,20240318000675,11011,2023,01082418,419530,CFS,연결재무제표,BS,재무상태표,이익잉여금,...,2023.12.31 현재,"-67,038,706,667",제 23 기,2022.12.31 현재,"-49,885,911,792",제 22 기,2021.12.31 현재,"-26,923,394,118",17,KRW
8,20240318000675,11011,2023,01082418,419530,CFS,연결재무제표,BS,재무상태표,자본총계,...,2023.12.31 현재,"47,091,518,931",제 23 기,2022.12.31 현재,"47,088,695,438",제 22 기,2021.12.31 현재,"-22,675,871,895",21,KRW
9,20240318000675,11011,2023,01082418,419530,CFS,연결재무제표,IS,손익계산서,매출액,...,2023.01.01 ~ 2023.12.31,"95,122,568,514",제 23 기,2022.01.01 ~ 2022.12.31,"68,322,017,040",제 22 기,2021.01.01 ~ 2021.12.31,"38,374,161,626",23,KRW
